In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
factories = pd.read_csv("../input/100honch6/tbl_factory.csv", index_col =0)
factories

In [ ]:
warehouses = pd.read_csv("../input/100honch6/tbl_warehouse.csv", index_col=0)
warehouses

In [ ]:
cost = pd.read_csv("../input/100honch6/rel_cost.csv", index_col=0)
cost

In [ ]:
trans = pd.read_csv("../input/100honch6/tbl_transaction.csv", index_col=0)
trans

In [ ]:
join_data = pd.merge(trans, cost, left_on = ["ToFC", "FromWH"], right_on = ["FCID", "WHID"], how="left")
join_data.head()

In [ ]:
join_data = pd.merge(join_data, factories, left_on="FCID", right_on="FCID", how="left")
join_data.head()

In [ ]:
join_data = pd.merge(join_data, warehouses, left_on="WHID", right_on="WHID", how="left")
join_data = join_data[["TransactionDate", "Quantity", "Cost", "ToFC", "FCName", "FCDemand", "FromWH", "WHName", "WHSupply", "WHRegion"]]
join_data.head()

In [ ]:
kanto = join_data.loc[join_data["WHRegion"]=="関東"]
kanto.head()

In [ ]:
tohoku = join_data.loc[join_data["WHRegion"]=="東北"]
tohoku.head()

In [ ]:
print("関東支社のコスト: " + str(kanto["Cost"].sum()) + "万円")
print("東北支社のコスト: " + str(tohoku["Cost"].sum()) + "万円")

In [ ]:
print("関東支社の総部品輸送個数" + str(kanto["Quantity"].sum()) + "個")
print("東北支社の総部品輸送個数" + str(tohoku["Quantity"].sum()) + "個")

In [ ]:
tmp = (kanto["Cost"].sum() / kanto["Quantity"].sum()) * 10000
print("関東支社の部品1つ当たりの輸送コスト: " + str(int(tmp)) + "円")

tmp = (tohoku["Cost"].sum() / tohoku["Quantity"].sum()) * 10000
print("東北支社の部品1つ当たりの輸送コスト: " + str(int(tmp)) + "円")

In [ ]:
cost_chk = pd.merge(cost, factories, on="FCID", how="left")
print("関東支社の平均輸送コスト: " + str(cost_chk["Cost"].loc[cost_chk["FCRegion"]=="関東"].mean()) + "万円")

print("東北支社の平均輸送コスト:" + str(cost_chk["Cost"].loc[cost_chk["FCRegion"]=="東北"].mean()) + "万円")

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

G=nx.Graph()

G.add_node("nodeA")
G.add_node("nodeB")
G.add_node("nodeC")

G.add_edge("nodeA", "nodeB")
G.add_edge("nodeA", "nodeC")
G.add_edge("nodeC", "nodeB")

pos={}
pos["nodeA"] = (0,0)
pos["nodeB"] = (1,1)
pos["nodeC"] = (0,1)

nx.draw(G,pos)

plt.show()

In [ ]:
G.add_node("nodeD")
G.add_edge("nodeA", "nodeD")
pos["nodeD"] = (1,0)

nx.draw(G,pos, with_labels=True)

In [ ]:
df_w = pd.read_csv('../input/100honch6/network_weight.csv')
df_p = pd.read_csv('../input/100honch6/network_pos.csv')

size = 10
edge_weights = []

for i in range(len(df_w)):
    for j in range(len(df_w.columns)):
        edge_weights.append(df_w.iloc[i][j] * size)
        
G = nx.Graph()

for i in range(len(df_w.columns)):
    G.add_node(df_w.columns[i])
    
for i in range(len(df_w.columns)):
    for j in range(len(df_w.columns)):
        G.add_edge(df_w.columns[i], df_w.columns[j])
        
        
pos={}
for i in range(len(df_w.columns)):
    node = df_w.columns[i]
    pos[node] = (df_p[node][0], df_p[node][1])
    
nx.draw(G, pos, with_labels=True, font_size=16, node_size=1000, node_color='k', font_color='w', width=edge_weights)

plt.show()

In [ ]:
df_tr = pd.read_csv("../input/100honch6/trans_route.csv", index_col="工場")
df_tr.head()

In [ ]:
df_pos = pd.read_csv("../input/100honch6/trans_route_pos.csv")
df_pos.head()

In [ ]:
# グラフオブジェクトの作成

G = nx.Graph()

# 頂点の設定

for i in range(len(df_pos.columns)):
    G.add_node(df_pos.columns[i])

# 辺の設定と、エッジの重みのリスト化

num_pre = 0
edge_weights= []
size = 0.1

for i in range(len(df_pos.columns)):
    for j in range(len(df_pos.columns)):
        if not (i==j):
            
            G.add_edge(df_pos.columns[i], df_pos.columns[j])
            
            if num_pre<len(G.edges):
                num_pre = len(G.edges)
                weight = 0
                if(df_pos.columns[i] in df_tr.columns)and(df_pos.columns[j] in df_tr.index):
                    if df_tr[df_pos.columns[i]][df_pos.columns[j]]:
                        weight = df_tr[df_pos.columns[i]][df_pos.columns[j]]*size
                elif(df_pos.columns[j] in df_tr.columns)and(df_pos.columns[i] in df_tr.index):
                    if df_tr[df_pos.columns[j]][df_pos.columns[i]]:
                        weight = df_tr[df_pos.columns[j]][df_pos.columns[i]]*size
                edge_weights.append(weight)

# 座標の設定
pos = {}
for i in range(len(df_pos.columns)):
    node = df_pos.columns[i]
    pos[node] = (df_pos[node][0], df_pos[node][1])


# 描画、表示
nx.draw(G, pos, with_labels=True, font_size=16, node_size=1000, node_color='k', font_color='w', width=edge_weights)
plt.show()


In [ ]:
df_tc = pd.read_csv("../input/100honch6/trans_cost.csv", index_col="工場")
df_tc.head()

In [ ]:
df_tr.head()

In [ ]:
#輸送コスト関数

def trans_cost(df_tr, df_tc):
    cost = 0
    for i in range(len(df_tr.index)):
        for j in range(len(df_tc.columns)):
            cost += df_tr.iloc[i][j] * df_tc.iloc[i][j]
    return cost

print("総輸送コスト:"+str(trans_cost(df_tr,df_tc)))

In [ ]:
df_demand = pd.read_csv("../input/100honch6/demand.csv")
df_supply = pd.read_csv("../input/100honch6/supply.csv")

#需要側の制約条件

for i in range(len(df_demand.columns)):
    tmp = sum(df_tr[df_demand.columns[i]])
    print(str(df_demand.columns[i])+"への輸送量:"+str(tmp)+"(需要量:" + str(df_demand.iloc[0][i]) + ")")
    if tmp>=df_demand.iloc[0][i]:
        print("需要量を満たしています。")
    else:
        print("需要量を満たしていません。輸送ルートを再計算してください。")

#供給側の制約条件

for i in range(len(df_supply.columns)):
    tmp = sum(df_tr.loc[df_supply.columns[i]])
    print(str(df_supply.columns[i]) + "への輸送量:" + str(tmp) + "(供給限界" + str(df_supply.iloc[0][i]) + ")")
    if tmp <= df_supply.iloc[0][i]:
        print("供給限界の範囲内です")
    else:
        print("供給限界を超過しています。輸送ルートを再計算してください。")

In [ ]:
df_tr_new = pd.read_csv("../input/100honch6/trans_route_new.csv", index_col="工場")
print(df_tr_new)

In [ ]:
print("総輸送コスト(変更後): " + str(trans_cost(df_tr_new, df_tc)))

In [ ]:
#制約条件を計算する関数を作成

#需要側
def condition_demand(df_tr, df_demand):
    flag = np.zeros(len(df_demand.columns))
    for i in range(len(df_demand.columns)):
        tmp = sum(df_tr[df_demand.columns[i]])
        if tmp >= df_demand.iloc[0][i]:
            flag[i] = 1
    return flag
    
#供給側
def condition_supply(df_tr, df_supply):
    flag = np.zeros(len(df_supply.columns))
    for i in range(len(df_supply.columns)):
        tmp = sum(df_tr.loc[df_supply.columns[i]])
        if tmp <= df_supply.iloc[0][i]:
            flag[i] = 1
    return flag


print("需要条件計算結果:" + str(condition_demand(df_tr_new, df_demand)))
print("供給条件計算結果:" + str(condition_supply(df_tr_new, df_supply)))
